# **Arxiv metadata Analytics with PySpark DF: JSON case study**

In [1]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=54b5eb59242b060c3d21c123485dc21f130168c863aedb78c8573f92433ad487
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
########## ONLY in Ubuntu Machine ##########
# Load Spark engine
!pip3 install -q findspark
import findspark
findspark.init()
########## ONLY in Ubuntu Machine ##########

In [2]:
# import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

spark

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Read and Load Data to Spark
df = spark.read.json("/content/drive/MyDrive/arxiv-metadata-oai-snapshot.json")
df.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_parsed: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- categories: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- journal-ref: string (nullable = true)
 |-- license: string (nullable = true)
 |-- report-no: string (nullable = true)
 |-- submitter: string (nullable = true)
 |-- title: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)



In [5]:
# check the partitions
df.rdd.getNumPartitions()

25

## Question 1: Create a new Schema

In [6]:
from pyspark.sql.types import *

Schema = StructType([
                    StructField('authors', StringType(), True),
                    StructField('categories', StringType(), True),
                    StructField('license', StringType(), True),
                    StructField('comments', StringType(), True),
                    StructField('abstract', StringType(), True),
                    StructField('versions', ArrayType(StringType()), True),
])

print(Schema)

StructType([StructField('authors', StringType(), True), StructField('categories', StringType(), True), StructField('license', StringType(), True), StructField('comments', StringType(), True), StructField('abstract', StringType(), True), StructField('versions', ArrayType(StringType(), True), True)])


## Question 2: Binding Data to a Schema

In [7]:
# if you provide schema the reading of data will be faster than default

df = spark.read.json("/content/drive/MyDrive/arxiv-metadata-oai-snapshot.json", schema = Schema)
df.show()

+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|             authors|       categories|             license|            comments|            abstract|            versions|
+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|C. Bal\'azs, E. L...|           hep-ph|                NULL|37 pages, 15 figu...|  A fully differe...|[{"version":"v1",...|
|Ileana Streinu an...|    math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|  We describe a n...|[{"version":"v1",...|
|         Hongjun Pan|   physics.gen-ph|                NULL| 23 pages, 3 figures|  The evolution o...|[{"version":"v1",...|
|        David Callan|          math.CO|                NULL|            11 pages|  We show that a ...|[{"version":"v1",...|
|Wael Abu-Shammala...|  math.CA math.FA|                NULL|                NULL|  In this paper w...|[{"version":"v1",...|


## Question 3: Missing values for "comments" and "license" attributes

In [8]:
# drop
df = df.dropna(subset = ["comments"])

# replace
df = df.fillna(value = "unknown", subset = ["license"])

df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             authors|          categories|             license|            comments|            abstract|            versions|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|C. Bal\'azs, E. L...|              hep-ph|             unknown|37 pages, 15 figu...|  A fully differe...|[{"version":"v1",...|
|Ileana Streinu an...|       math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|  We describe a n...|[{"version":"v1",...|
|         Hongjun Pan|      physics.gen-ph|             unknown| 23 pages, 3 figures|  The evolution o...|[{"version":"v1",...|
|        David Callan|             math.CO|             unknown|            11 pages|  We show that a ...|[{"version":"v1",...|
|Y. H. Pong and C....|   cond-mat.mes-hall|             unknown|6 pages, 4 figure...|  We study the tw..

## Question 4: Get the author names who published a paper in a 'math' category

In [10]:
df.createOrReplaceTempView("arxiv")

query = """
SELECT authors
FROM arxiv
WHERE categories LIKE 'math%'
"""

spark.sql(query).show()

print("Number of rows: ", spark.sql(query).count())

+--------------------+
|             authors|
+--------------------+
|Ileana Streinu an...|
|        David Callan|
|  Sergei Ovchinnikov|
|Clifton Cunningha...|
|        Koichi Fujii|
|         Norio Konno|
|Simon J.A. Malham...|
|Robert P. C. de M...|
|  P\'eter E. Frenkel|
|          Mihai Popa|
|   Debashish Goswami|
|      Mikkel {\O}bro|
|Nabil L. Youssef,...|
|         Boris Rubin|
|         A. I. Molev|
| Branko J. Malesevic|
|   John W. Robertson|
|     Yu.N. Kosovtsov|
|        Osamu Fujino|
|Stephen C. Power ...|
+--------------------+
only showing top 20 rows

Number of rows:  304590


## Question 5: Get linceses with 5 or more letters in the "abstract"

In [13]:
sql_query = """ SELECT distinct(license) FROM arxiv
                WHERE abstract REGEXP '%\(([A-Za-z][^_ /\\<>]{5,})\)%'
            """


spark.sql(sql_query).show()
print("Number of rows: ", spark.sql(sql_query).count())


+--------------------+
|             license|
+--------------------+
|http://arxiv.org/...|
|http://creativeco...|
|http://creativeco...|
|http://creativeco...|
|             unknown|
+--------------------+

Number of rows:  5


## Question 6: Extract the statistic of the number of pages for unknown licenses

In [16]:
import re

def get_pages(line):
  search = re.findall(r"\d+ pages", line)
  if search:
    return int(search[0].split(" ")[0])
  else:
    return 0


get_pages('there are 100 pages')

100

**Register the UDF**

In [17]:
spark.udf.register("get_pages", get_pages)

<function __main__.get_pages(line)>

In [20]:
query = """SELECT AVG(get_pages(comments)) AS avg, SUM(get_pages(comments)) AS sum,
                STD(get_pages(comments)) AS std
                FROM arxiv
                WHERE license="unknown"
                """

In [21]:
spark.sql(query).show()

+------------------+---------+------------------+
|               avg|      sum|               std|
+------------------+---------+------------------+
|13.368011068572079|5642584.0|16.777518213632323|
+------------------+---------+------------------+

